In [ ]:
import pandas as pd  # Importa la libreria pandas per la manipolazione dei dati
import sqlite3       # Importa la libreria sqlite3 per lavorare con database SQLite

conn = sqlite3.connect("azienda_dipendenti_aggiornato.db")  # Crea una connessione al database SQLite

dipendenti_df = pd.DataFrame({  # Crea un DataFrame pandas con i dati dei dipendenti
    'id': range(1, 9),
    'nome': ['Marco', 'Anna', 'Luca', 'Sara', 'Giulia', 'Paolo', 'Elena', 'Franco'],
    'dipartimento': ['IT', 'HR', 'IT', 'Marketing', 'HR', 'IT', 'Marketing', 'Contabilità'],
    'stipendio': [2000, 1800, 2200, 1900, 1750, 2100, 1950, 1850],
    'data_assunzione': [
        '2018-05-10', '2021-06-15', '2019-07-01', '2022-01-10',
        '2017-03-12', '2015-11-30', '2023-04-25', '2016-09-20'
    ]
})

dipendenti_df.to_sql('dipendenti', conn, if_exists='replace', index=False)  # Salva il DataFrame come tabella 'dipendenti' nel database

media_dipartimenti = pd.read_sql_query("""  # Esegue una query SQL per calcolare la media degli stipendi per dipartimento
SELECT dipartimento, AVG(stipendio) AS media_stipendio
FROM dipendenti
GROUP BY dipartimento
""", conn)
print("Media stipendio per dipartimento:")  # Stampa una descrizione
print(media_dipartimenti)                   # Stampa il risultato della query

media_dipartimenti = pd.read_sql_query("""  # Ripete la stessa query di prima (ridondante)
SELECT dipartimento, AVG(stipendio) AS media_stipendio
FROM dipendenti
GROUP BY dipartimento
""", conn)
print("Media stipendio per dipartimento:")  # Stampa una descrizione
print(media_dipartimenti)                   # Stampa il risultato della query

conn.execute("""  # Aggiorna lo stipendio a 2000 per i dipendenti assunti prima del 2020
UPDATE dipendenti
SET stipendio = 2000
WHERE data_assunzione < '2020-01-01'
""")
conn.commit()  # Salva le modifiche nel database

sopra_media_df = pd.read_sql_query("""  # Seleziona i dipendenti con stipendio sopra la media generale (prima versione)
SELECT *
FROM dipendenti
WHERE stipendio > (
    SELECT AVG(stipendio)
    FROM dipendenti
)
""", conn)

media_generale = pd.read_sql_query("""  # Calcola la media generale degli stipendi
SELECT AVG(stipendio) AS media_generale FROM dipendenti
""", conn).iloc[0, 0]

sopra_media_df = pd.read_sql_query(f"""  # Seleziona i dipendenti con stipendio sopra la media generale (usando la variabile)
SELECT * FROM dipendenti
WHERE stipendio > {media_generale}
""", conn)

print("\nDipendenti con stipendio sopra la media generale:")  # Stampa una descrizione
print(sopra_media_df)                                         # Stampa il risultato della query

conn.close()  # Chiude la connessione al database

Media stipendio per dipartimento:
  dipartimento  media_stipendio
0  Contabilità           1850.0
1           HR           1775.0
2           IT           2100.0
3    Marketing           1925.0
Media stipendio per dipartimento:
  dipartimento  media_stipendio
0  Contabilità           1850.0
1           HR           1775.0
2           IT           2100.0
3    Marketing           1925.0

Dipendenti con stipendio sopra la media generale:
   id    nome dipartimento  stipendio data_assunzione
0   1   Marco           IT       2000      2018-05-10
1   3    Luca           IT       2000      2019-07-01
2   5  Giulia           HR       2000      2017-03-12
3   6   Paolo           IT       2000      2015-11-30
4   8  Franco  Contabilità       2000      2016-09-20
